动态网页的爬取

In [6]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import json
import re
import multiprocessing
# import xlwt
import time

In [3]:
# 手动找到真实的request headers

wangyi = 'https://news.163.com/special/cm_yaowen20200213_04/?callback=data_callback'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                        'AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/92.0.4515.107 Safari/537.36 Edg/92.0.902.55'}
response = requests.get(wangyi, headers=header)
print(response.status_code)

200


In [9]:
# data = response.text
print(response.text)

data_callback([
                        
                        
                                                            
                                                                     {
        "title":"“9·11”纪念日当天拜登喊话美军：我们欠你们的",
        "digest":"",
        "docurl":"https://www.163.com/dy/article/HH182KHJ0514D3UH.html",
        "commenturl":"https://comment.tie.163.com/HH182KHJ0514D3UH.html",
        "tienum":1169,
        "tlastid":"<a href='https://news.163.com' class='link link_more'><em>新闻</em></a>",
        "tlink":"https://www.163.com/dy/article/HH182KHJ0514D3UH.html",
        "label":"其它",
        "keywords":[
                                                ],
        "time":"09/12/2022 08:22:31",
        "newstype":"article",
        "pics3":[
                    ],
        "channelname":"yaowen",
        "source":"封面新闻",
        "point":"70",
                    "imgurl":"http://cms-bucket.ws.126.net/2022/0912/cb9e7e73p00ri2m7n005ec000hs00fcc.png",
            "a

# 逐步讲解

In [20]:
# response.text 不符合json格式，去掉最外层（）保留为列表

start = response.text.index('[')
end = response.text.index('])') + 1
print(start,end)

14 80314


In [18]:
print(response.text[start:end])

[
                        
                        
                                                            
                                                                     {
        "title":"“9·11”纪念日当天拜登喊话美军：我们欠你们的",
        "digest":"",
        "docurl":"https://www.163.com/dy/article/HH182KHJ0514D3UH.html",
        "commenturl":"https://comment.tie.163.com/HH182KHJ0514D3UH.html",
        "tienum":1169,
        "tlastid":"<a href='https://news.163.com' class='link link_more'><em>新闻</em></a>",
        "tlink":"https://www.163.com/dy/article/HH182KHJ0514D3UH.html",
        "label":"其它",
        "keywords":[
                                                ],
        "time":"09/12/2022 08:22:31",
        "newstype":"article",
        "pics3":[
                    ],
        "channelname":"yaowen",
        "source":"封面新闻",
        "point":"70",
                    "imgurl":"http://cms-bucket.ws.126.net/2022/0912/cb9e7e73p00ri2m7n005ec000hs00fcc.png",
            "add1":"",
     

In [17]:
# 使用json转换成python数据格式

data = json.loads(response.text[start:end])
print(data)

[{'title': '“9·11”纪念日当天拜登喊话美军：我们欠你们的', 'digest': '', 'docurl': 'https://www.163.com/dy/article/HH182KHJ0514D3UH.html', 'commenturl': 'https://comment.tie.163.com/HH182KHJ0514D3UH.html', 'tienum': 1169, 'tlastid': "<a href='https://news.163.com' class='link link_more'><em>新闻</em></a>", 'tlink': 'https://www.163.com/dy/article/HH182KHJ0514D3UH.html', 'label': '其它', 'keywords': [], 'time': '09/12/2022 08:22:31', 'newstype': 'article', 'pics3': [], 'channelname': 'yaowen', 'source': '封面新闻', 'point': '70', 'imgurl': 'http://cms-bucket.ws.126.net/2022/0912/cb9e7e73p00ri2m7n005ec000hs00fcc.png', 'add1': '', 'add2': '', 'add3': ''}, {'title': '英国示威者抗议黑人歌手遭枪击 英媒：他们来悼念女王', 'digest': '', 'docurl': 'https://www.163.com/dy/article/HH10BSLG05504DPG.html', 'commenturl': 'https://comment.tie.163.com/HH10BSLG05504DPG.html', 'tienum': 913, 'tlastid': "<a href='https://news.163.com' class='link link_more'><em>新闻</em></a>", 'tlink': 'https://www.163.com/dy/article/HH10BSLG05504DPG.html', 'label': '其它', 'k

In [32]:
# 数据清洗 + 二次爬取（点击链接，爬取新闻内容）

for item in data:
    # print(item['title'])
    # print(item['time'])
    # print(item['docurl'])
    # print(item['source'])
    inurl = item['docurl']
    print(inurl)
    inresponse = requests.get(inurl, headers=header).text

    # Xpath 不适合
    inhtml = etree.HTML(inresponse)
    print(inhtml.xpath('//div[@class="post_body"]/text()'))

    # BeautifulSoup
    bs = BeautifulSoup(inresponse, 'lxml')
    news = bs.findAll('div', class_='post_body')[0].text
    print(news)

    break

# data = json.loads(response.text)
# print(data)

https://www.163.com/dy/article/HH182KHJ0514D3UH.html
['\n                    ', '\n                ']

来源：封面新闻封面新闻记者 燕磊 综合编译报道“我们永远不会忘记”，当地时间11日上午，美国总统拜登发文纪念美国“9·11”事件21周年。21年前的今天，恐怖分子劫持四架民航客机冲向纽约世贸中心及五角大楼，将纽约地标世贸中心双子塔从城市天际线彻底抹除。世界贸易中心建造于1970年，外墙为玻璃幕墙，共28层，地上26层，地下2层。大厦里有公司1200百多个，员工达50多万人。资料图拜登：“一笔永远无法偿还的债务”据美国五角大楼官网消息，当地时间11日9时许，美国总统拜登出席在五角大楼“9·11”纪念馆举行的“9·11”事件21周年纪念仪式，美国国防部长奥斯汀、美军参谋长联席会议主席米利也出席了该活动。拜登当天在讲话中表示，“对于我们所有的军人和他们的家人，他们为我们的国家做出了如此多的牺牲，我们欠你们的，我们欠你们一笔巨大的债务。一笔永远无法偿还的债务。”拜登表示，“21年后，我们对在世贸中心遗址、宾夕法尼亚州尚克斯维尔和五角大楼被夺走的所有宝贵生命记忆犹新。对于那些如今仍然感到痛苦的遇难者家属，吉尔（第一夫人）和我在心里和你们紧紧地抱在一起。”据悉，此次“9·11”事件21周年纪念仪式未向公众开放，仅允许遇难者家属前往哀悼。资料图“9·11”后不到一个月阿富汗战争开始美国官方资料显示，“9·11”事件夺去2996人的生命。就在“9·11”事件发生后不到一个月，美国以“反恐”之名，发起了阿富汗战争。最初的10年，美军在阿富汗战场深陷与塔利班游击队周旋的泥潭中。10年后，美军开始计划撤军，随后花了10年时间最终完成撤离。直到2021年8月末，美军宣布从阿富汗完成撤军。持续近20年的阿富汗战争，给美国人和阿富汗人都带来巨大损失。据美国布朗大学统计，20年阿富汗战争中，美国付出了2400多名军人死亡和超过2万亿美元损失的代价。阿富汗累计有3万多名平民因战乱而死亡，另有6万多名平民受伤，约1100万人沦为难民。长年战乱导致阿富汗经济凋敝，约72%的民众生活在贫困线以下，失业率高达38%。资料图21年来美国人并未停止担心“9·11”事件后，美国军费进入增长轨道，联邦政府陷入财

# 合并
思路
1. 抓取每次翻页的页面链接
2. 每一页各新闻的标题
3. 每一新闻url跳转后的内容

In [32]:
def PageUrl():
